**Домашнее задание**

1. скачать набор данных маркетинговых кампаний отсюда

[https://www.kaggle.com/davinwijaya/customer-retention](https://www.kaggle.com/davinwijaya/customer-retention)

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv

In [9]:
df = pd.read_csv('data.csv')
df['conversion'].value_counts(), df['offer'].value_counts()

(0    54606
 1     9394
 Name: conversion, dtype: int64,
 Buy One Get One    21387
 Discount           21307
 No Offer           21306
 Name: offer, dtype: int64)

2. там поле conversion - это целевая переменная, а offer - коммуникация. Переименовать поля (conversion -> target, offer -> treatment) и привести поле treatment к бинарному виду (1 или 0, т.е было какое-то предложение или нет) - значение No Offer означает отсутствие коммуникации, а все остальные - наличие.

In [36]:
rename_col = {'conversion': 'target', 'offer': 'treatment'}
df.rename(columns=rename_col, inplace=True)
df.head(3)

,recency,history,used_discount,used_bogo,zip_code,is_referral,channel,treatment,target
0,10,142.44,1,0,Surburban,0,Phone,1,0
1,6,329.08,1,1,Rural,1,Web,0,0
2,7,180.65,0,1,Surburban,1,Web,1,0


In [26]:
treat_map = {'Buy One Get One': 1, 'Discount': 1, 'No Offer': 0}
df['offer'] = df['offer'].map(treat_map)

In [29]:
df.offer.value_counts()

1    42694
0    21306
Name: offer, dtype: int64

3. сделать разбиение набора данных не тренировочную и тестовую выборки

In [44]:
from sklearn.model_selection import train_test_split
pd.set_option('display.max_columns', None)

X = df[df.columns[:-1]]
y = df[df.columns[-1]]

X_train, y_train, X_test, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

4. сделать feature engineering на ваше усмотрение (допускается свобода выбора методов)

In [46]:
df.zip_code.value_counts()

Surburban    28776
Urban        25661
Rural         9563
Name: zip_code, dtype: int64

In [49]:
df

,recency,history,used_discount,used_bogo,zip_code,is_referral,channel,treatment,target
0,10,142.44,1,0,Surburban,0,Phone,1,0
1,6,329.08,1,1,Rural,1,Web,0,0
2,7,180.65,0,1,Surburban,1,Web,1,0
3,9,675.83,1,0,Rural,1,Web,1,0
4,2,45.34,1,0,Urban,0,Web,1,0
...,...,...,...,...,...,...,...,...,...
63995,10,105.54,1,0,Urban,0,Web,1,0
63996,5,38.91,0,1,Urban,1,Phone,1,0
63997,6,29.99,1,0,Urban,1,Phone,1,0
63998,1,552.94,1,0,Surburban,1,Multichannel,1,0


In [50]:
base_columns = ['used_discount', 'used_bogo', 'is_referral', 'treatment']
cat_columns = ['zip_code', 'channel']
real_columns = ['recency', 'history']

In [59]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.compose import make_column_selector, make_column_transformer



encoder = LabelEncoder()

ct = make_column_transformer(
                (StandardScaler(), 
                make_column_selector(df[real_columns])),
                (LabelEncoder(), 
                make_column_selector(df[cat_columns])))

ct.fit_transform(X_train)

TypeError: 'DataFrame' objects are mutable, thus they cannot be hashed

5. провести uplift-моделирование 3 способами: одна модель с признаком коммуникации (S learner), модель с трансформацией таргета (трансформация классов п. 2. 1) и вариант с двумя независимыми моделями

6. в конце вывести единую таблицу сравнения метрик uplift@10%, uplift@20% этих 3 моделей

7. построить модель UpliftTreeClassifier и попытаться описать словами полученное дерево

8. (опционально) для модели S learner (модель с дополнительным признаком коммуникации) построить зависимость таргета (конверсии - поле conversion) от значения uplift: 1) сделать прогноз и получить uplift для тестовой выборки 2) отсортировать тестовую выборку по uplift по убыванию 3) разбить на децили (pandas qcut вам в помощь) 4) для каждого дециля посчитать среднюю conversion

9. (опционально) построить модель UpliftRandomForestClassifier и попытаться описать словами полученное дерево

**Ссылки**

[https://towardsdatascience.com/a-quick-uplift-modeling-introduction-6e14de32bfe0](https://towardsdatascience.com/a-quick-uplift-modeling-introduction-6e14de32bfe0)

[https://habr.com/ru/company/ru_mts/blog/485980/#reference1](https://habr.com/ru/company/ru_mts/blog/485980/#reference1)

[https://en.wikipedia.org/wiki/Uplift_modelling](https://en.wikipedia.org/wiki/Uplift_modelling)

[https://www.youtube.com/watch?v=yFQAIJBYXI0](https://www.youtube.com/watch?v=yFQAIJBYXI0)

[https://www.youtube.com/watch?v=jCUcYiBK03I](https://www.youtube.com/watch?v=jCUcYiBK03I)

[https://www.uplift-modeling.com/en/latest/](https://www.uplift-modeling.com/en/latest/)

[https://arxiv.org/pdf/1809.04559.pdf](https://arxiv.org/pdf/1809.04559.pdf)

[https://catboost.ai/docs/concepts/about.html](https://catboost.ai/docs/concepts/about.html)

**Библиотеки и пакеты**

causalml

sklift

catboost